In [ ]:
import importlib
import math
import random

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
import torch

from calibration import data
from calibration import dists
from calibration import mdn
from calibration import plot
from calibration import vae
from calibration import uci

## Data Sets

In [ ]:
BINS = 20

In [ ]:
torch.manual_seed(78)
TESTS = 100
testset = data.PITHistRndDataset(TESTS, BINS, samples=100)
len(testset)

In [ ]:
refset = data.PITHistRefDataset(BINS, steps=10)
len(refset)

## Mixture Density Network

In [ ]:
checkpoint = torch.load("models/iconic-firebrand-685.pt")
hyperparams = checkpoint["hyperparams"]
model = mdn.MDN(inputs=hyperparams["bins"],
                neurons=hyperparams["neurons"],
                hiddens=hyperparams["hiddens"],
                k=hyperparams["k"])
model.load_state_dict(checkpoint["model_state_dict"])
model

In [ ]:
alpha, mu, sigma = model.predict(testset.X)

In [ ]:
for i in range(len(testset)):
    dist_data = data.y2dist(*testset.y[i])
    dist_mdn = mdn.mdn2dist(alpha[i], mu[i], sigma[i])
    i = euclidean_distances(testset.X[i].unsqueeze(0), refset.X).argmin()
    dist_nn = data.y2dist(*refset.y[i])
    # visualisation
    _, ax = plt.subplots()
    plot.pdf(ax, dist_data, label="data generating")
    plot.pdf(ax, dist_mdn, label="prediction")
    plot.pdf(ax, dist_nn, label="nearest neighbor")
    ax.legend()
    plt.show()
    plt.close()

## Uniform PIT histogram

In [ ]:
pit_hist_uniform = torch.full((BINS, ), 1 / BINS)
alpha_uniform, mu_uniform, sigma_uniform = model.predict(pit_hist_uniform)
dist_uniform = mdn.mdn2dist(alpha_uniform, mu_uniform, sigma_uniform)
_, ax = plt.subplots()
plot.pdf(ax, dists.Normal(0, 1), label="data generating")
plot.pdf(ax, dist_uniform, label="prediction")
ax.legend()
dist_uniform

## Protein Data Set

In [ ]:
(_, proteinset), _ = uci.get_dataset("protein", seed=50, validation=False, preparation=True)
X_protein, y_protein = proteinset.tensors
nn = uci.NeuralNetwork(X_protein.shape[-1], {"loss": "nll", "neurons": 64, "hiddens": 1})
nn.load("nll-1-1-5432")
y_pred_protein = nn.predict(proteinset)
pit_values_protein = uci.normal_pit(*y_pred_protein, y_protein.cpu()).squeeze()
pit_hist_protein = data.pit_hist(pit_values_protein, BINS)

In [ ]:
alpha_protein, mu_protein, simga_protein = model.predict(pit_hist_protein)

In [ ]:
alpha_protein

In [ ]:
sample = dists.sample_normal_mixture(alpha_protein, mu_protein, simga_protein, 10000)
pit_values = data.pit(dists.Normal(0, 1), sample)
pit_hist = data.pit_hist(pit_values, BINS)

In [ ]:
dist_protein = mdn.mdn2dist(alpha_protein, mu_protein, simga_protein)
_, (ax1, ax2) = plt.subplots(nrows=2)
plot.pit_hist(ax1, pit_hist_protein, BINS)
plot.pit_hist(ax1, pit_hist, BINS)
plot.pdf(ax2, dist_protein)
dist_protein